### DQN Docking Case

In [1]:
# setup
import gym
import gym_space_docking

env = gym.make('space_docking-v0')


pygame 2.1.0 (SDL 2.0.16, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: No file 'assets/med_ship_01.png' found in working directory '/home/meyd/Documents/GitHub/Denny-Meyer/IronHack_Final_Project/python'.

### create enviroment pygame

In [ ]:
!pip install -e ./gym_space_docking/

Error: Session cannot generate requests

In [ ]:
# create network

